In [1]:
import os

import torch

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Using device:", device)

from TTS.utils.manage import ModelManager
from TTS.api import TTS
import TTS.tts.configs.xtts_config as xtts_cfg
from IPython.display import Audio

Using device: mps


In [2]:
out_dir = "./example_output"
os.makedirs(out_dir, exist_ok=True)

In [3]:
# manager = ModelManager()
# models = manager.list_models()
# print(models)

In [4]:
# Text was generated by chatGPT
text = """
The morning sun streamed through the window, casting warm golden light across the room. Birds chirped cheerfully outside, adding a lively soundtrack to the quiet start of the day. It was the perfect moment to enjoy a cup of coffee and plan the day ahead.
"""

In [5]:
filename = os.path.join(out_dir, "xtts_v2.wav")
speaker_wav = "./speakers/wavs/LJ001-0001.wav"
tts = TTS("tts_models/multilingual/multi-dataset/your_tts")
tts.tts_to_file(text=text, file_path=filename, language="en", speaker_wav=speaker_wav)

 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-

'./example_output/xtts_v2.wav'

In [6]:
Audio(filename)

In [7]:
import noisereduce as nr
import soundfile as sf
import numpy as np
from scipy.signal import fftconvolve, butter, lfilter
# import nara_wpe
from nara_wpe.wpe import wpe
from scipy.signal import stft, istft
    

# 2️⃣ Echo / reverb reduction
# Simple high-pass filter to reduce low-frequency rumble
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def highpass_filter(data, cutoff, fs):
    b, a = butter_highpass(cutoff, fs)
    y = lfilter(b, a, data)
    return y

def denoise_file(input_path, output_path):
    data, sr = sf.read(input_path)

    # If stereo, convert to mono
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)

    # 1️⃣ Noise reduction using noisereduce
    # You can capture a small segment of silence to estimate noise
    noise_sample = data[0:int(sr*0.5)]  # first 0.5 sec
    denoised = nr.reduce_noise(y=data, y_noise=noise_sample, sr=sr)

    cleaned = highpass_filter(denoised, cutoff=80, fs=sr)  # adjust cutoff if needed
    
    # Save the cleaned audio
    sf.write(output_path, cleaned, sr)

    print("Audio cleaned and saved as tts_cleaned.wav")
    return sr


In [8]:
denoised_file_path = "./denoised/morning_sun.wav"
denoise_file(filename, denoised_file_path)

Audio cleaned and saved as tts_cleaned.wav


16000

In [9]:
Audio(denoised_file_path)